In [1]:
%matplotlib widget

from pathlib import Path 

import numpy as np
import matplotlib.pyplot as plt

from skimage.color import label2rgb
from skimage.filters import sobel
from skimage import morphology
import skimage.segmentation as seg
from scipy import ndimage as ndi

import ncempy.io as nio

import ipywidgets as widgets

## Load data

In [2]:
dPath = Path(r'C:\Users\alexl\Documents\Python\sphere-finder\data')
fPath = 'CaCO3_HAADF_2.emd'

emd1 = nio.emdVelox.fileEMDVelox(dPath / fPath)
#emd1['data']
print(emd1) # print information about the file
im0, metadata0 = emd1.get_dataset(0)
print(metadata0)

fg, ax = plt.subplots(1,1)
ax.imshow(im0, cmap='gray')

EMD file contains 1 data sets
Dataset #0 from detector: HAADF
pixel size = (5.4830, 5.4830) nm
{'pixelSizeUnit': ['nm', 'nm'], 'pixelSize': (5.483017815649768, 5.483017815649768), 'AcquisitionTime': datetime.datetime(2020, 11, 25, 16, 31, 46), 'Stage': {'Position': {'x': '-0.0001256090550000001', 'y': '-0.00051432961799999992', 'z': '-0.00011989836000000002'}, 'AlphaTilt': '-3.1359999999734965e-06', 'BetaTilt': '0.00055922558200545609', 'HolderType': 'FEI Double Tilt'}, 'detectorName': 'HAADF', 'dwellTime': '9.9999999999999995e-07'}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Image histogram

In [3]:
fig, ax = plt.subplots(1, 1)
ax.hist(im0.ravel(), bins=32, range=[20000, 45000])
ax.set_xlim(20000, 45000);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Region-based segmentation

In [4]:
#defining image data variable
im_data = im0

#Sobel gradient of image
elevation_map = sobel(im_data)
fg = plt.figure()
plt.imshow(elevation_map)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
#finding markers of the background based on extreme parts of the histogram
markers = np.zeros_like(im_data)
markers[im_data < 30000] = 1
markers[im_data > 32000] = 2

fg = plt.figure()
plt.imshow(markers)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
#watershed transform to fill regions of the elevation map starting from the markers
segmentation_image = seg.watershed(elevation_map, markers)
segmentation_image = ndi.binary_fill_holes(segmentation_image - 1)
fg = plt.figure()
plt.imshow(segmentation_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#removing small fragments and incomplete spheres
cleaned_image = morphology.remove_small_objects(segmentation_image, 10000)
labeled_spheres, number = ndi.label(cleaned_image)
image_label_overlay = label2rgb(labeled_spheres, image=im_data, bg_label=0)

fg, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
axes[0].imshow(im_data, cmap=plt.cm.gray)
axes[0].contour(cleaned_image, [0.5], linewidths=0.5, colors='y')
axes[1].imshow(image_label_overlay)

#display number of spheres in image
print(number)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2
